<a href="https://colab.research.google.com/github/duy7590/portfolio_tracker/blob/master/Investment_Portfolio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**References:**  

https://www.datacamp.com/community/tutorials/finance-python-trading

https://github.com/datacamp/datacamp-community-tutorials/blob/master/Python%20Finance%20Tutorial%20For%20Beginners/Python%20For%20Finance%20Beginners%20Tutorial.ipynb

https://pypi.python.org/pypi/fix-yahoo-finance

http://www.learndatasci.com/python-finance-part-yahoo-finance-api-pandas-matplotlib/

In [1]:
# Import initial libraries

import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Imports in order to be able to use Plotly offline.
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import plotly.graph_objs as go

print(__version__) # requires version >= 1.9.0

init_notebook_mode(connected=True)

4.9.0


In [3]:
# Import the Sample worksheet with acquisition dates and initial cost basis:

import pandas as pd
import io
import requests
url="https://raw.githubusercontent.com/duy7590/portfolio_tracker/master/data.csv"
s=requests.get(url).content
portfolio_df = pd.read_csv(io.StringIO(s.decode('utf-8')))
portfolio_df['Acquisition Date'] = portfolio_df['Acquisition Date'].astype('datetime64[ns]') 
portfolio_df['Start of Year'] = portfolio_df['Start of Year'].astype('datetime64[ns]') 
portfolio_df['Cost Basis'] = portfolio_df['Cost Basis'].astype('float64') 
portfolio_df['Unit Cost'] = portfolio_df['Unit Cost'].astype('float64') 
portfolio_df.head(10)

,Acquisition Date,Ticker,Quantity,Unit Cost,Cost Basis,Start of Year
0,2013-02-07,AAPL,125,65.40,8175.00,2019-12-31
1,2014-02-27,JNJ,100,81.90,8190.00,2019-12-31
2,2013-06-04,MCD,100,84.99,8499.00,2019-12-31
3,2015-12-14,MTCH,600,13.63,8178.00,2019-12-31
4,2016-01-14,NFLX,75,108.71,8153.25,2019-12-31
5,2013-08-14,WMT,125,68.30,8537.50,2019-12-31
6,2013-12-13,FB,150,53.32,7998.00,2019-12-31
7,2015-01-05,TWTR,225,36.38,8185.50,2019-12-31


In [4]:
# Confirm that you have 8 values for each column.

portfolio_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 6 columns):
Acquisition Date    8 non-null datetime64[ns]
Ticker              8 non-null object
Quantity            8 non-null int64
Unit Cost           8 non-null float64
Cost Basis          8 non-null float64
Start of Year       8 non-null datetime64[ns]
dtypes: datetime64[ns](2), float64(2), int64(1), object(1)
memory usage: 456.0+ bytes


In [5]:
# Date Ranges for SP 500 and for all tickers
# Modify these date ranges each week.

# The below will pull back stock prices from 2010 until end date specified.
start_sp = datetime.datetime(2013, 1, 1)
end_sp = datetime.datetime(2020, 6, 6)

# This variable is used for YTD performance.
end_of_last_year = datetime.datetime(2019, 12, 31)

# These are separate if for some reason want different date range than SP.
stocks_start = datetime.datetime(2013, 1, 1)
stocks_end = datetime.datetime(2020, 6, 6)

In [6]:
end_of_last_year

datetime.datetime(2019, 12, 31, 0, 0)

In [7]:
# Leveraged from the helpful Datacamp Python Finance trading blog post.

from pandas_datareader import data as pdr
!pip install yfinance
import yfinance as yf
yf.pdr_override() # <== that's all it takes :-)

sp500 = pdr.get_data_yahoo('^GSPC', 
                           start_sp,
                             end_sp)
                          
sp500.head()

[*********************100%***********************]  1 of 1 completed


,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,
2013-01-02,1462.420044,1462.420044,1462.430054,1426.189941,1426.189941,4202600000
2013-01-03,1459.369995,1459.369995,1465.469971,1455.530029,1462.420044,3829730000
2013-01-04,1466.469971,1466.469971,1467.939941,1458.989990,1459.369995,3424290000
2013-01-07,1461.890015,1461.890015,1466.469971,1456.619995,1466.469971,3304970000
2013-01-08,1457.150024,1457.150024,1461.890015,1451.640015,1461.890015,3601600000


In [8]:
sp500.tail()

,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,
2020-06-01,3055.729980,3055.729980,3062.179932,3031.540039,3038.780029,4673410000
2020-06-02,3080.820068,3080.820068,3081.070068,3051.639893,3064.780029,5187230000
2020-06-03,3122.870117,3122.870117,3130.939941,3098.899902,3098.899902,5989560000
2020-06-04,3112.350098,3112.350098,3128.909912,3090.409912,3111.560059,6428130000
2020-06-05,3193.929932,3193.929932,3211.719971,3163.840088,3163.840088,8617590000


In [9]:
# Create a dataframe with only the Adj Close column as that's all we need for this analysis.

sp_500_adj_close = sp500[['Adj Close']].reset_index()

In [10]:
sp_500_adj_close

,Date,Adj Close
0,2013-01-02,1462.420044
1,2013-01-03,1459.369995
2,2013-01-04,1466.469971
3,2013-01-07,1461.890015
4,2013-01-08,1457.150024
5,2013-01-09,1461.020020
6,2013-01-10,1472.119995
7,2013-01-11,1472.050049
8,2013-01-14,1470.680054
9,2013-01-15,1472.339966


In [11]:
# Adj Close for the EOY in 2017 in order to run comparisons versus stocks YTD performances.

sp_500_adj_close_start = sp_500_adj_close[sp_500_adj_close['Date']==end_of_last_year]
sp_500_adj_close_start

,Date,Adj Close
1761,2019-12-31,3230.780029


In [12]:
# Generate a dynamic list of tickers to pull from Yahoo Finance API based on the imported file with tickers.
tickers = portfolio_df['Ticker'].unique()
tickers

array(['AAPL', 'JNJ', 'MCD', 'MTCH', 'NFLX', 'WMT', 'FB', 'TWTR'],
      dtype=object)

In [13]:
# Stock comparison code

def get(tickers, startdate, enddate):
    def data(ticker):
        return (pdr.get_data_yahoo(ticker, start=startdate, end=enddate))
    datas = map(data, tickers)
    return(pd.concat(datas, keys=tickers, names=['Ticker', 'Date']))
               
all_data = get(tickers, stocks_start, stocks_end)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [55]:
all_data

Adj Close      Close  ...       Open     Volume
Ticker Date                              ...                      
AAPL   2013-01-02  68.502106  78.432854  ...  79.117142  140129500
       2013-01-03  67.637436  77.442856  ...  78.268570   88241300
       2013-01-04  65.753418  75.285713  ...  76.709999  148583400
       2013-01-07  65.366646  74.842857  ...  74.571426  121039100
       2013-01-08  65.542587  75.044289  ...  75.601425  114676800
       2013-01-09  64.518211  73.871429  ...  74.642860  101901100
       2013-01-10  65.317978  74.787140  ...  75.507141  150286500
       2013-01-11  64.917465  74.328575  ...  74.428574   87626700
       2013-01-14  62.602997  71.678574  ...  71.811432  183551900
       2013-01-15  60.627903  69.417145  ...  71.185715  219193100
       2013-01-16  63.144505  72.298569  ...  70.662857  172701200
       2013-01-17  62.719036  71.811432  ...  72.901428  113419600
       2013-01-18  62.384659  71.428574  ...  71.217140  118230700
       2013-01-22  62.979813  72.110001  ...  72.080002  115386600
       2013-01-23  64.132660  73.430000  ...  72.687141  215377400
       2013-01-24  56.208576  64.357140  ...  65.714287  365213100
       2013-01-25  54.883522  62.840000  ...  64.527145  302006600
       2013-01-28  56.124973  64.261429  ...  62.547142  196379400
       2013-01-29  57.178040  65.467140  ...  65.500000  142789500
       2013-01-30  56.998371  65.261429  ...  65.285713  104288800
       2013-01-31  56.831169  65.070000  ...  65.282860   79833600
       2013-02-01  56.597866  64.802856  ...  65.587143  134871100
       2013-02-04  55.187954  63.188572  ...  64.844284  119279300
       2013-02-05  57.124386  65.405716  ...  63.435715  143336900
       2013-02-06  57.063232  65.335716  ...  65.209999  148426600
       2013-02-07  58.759972  66.888573  ...  66.178574  176145200
       2013-02-08  59.608299  67.854286  ...  67.714287  158289600
       2013-02-11  60.229511  68.561432  ...  68.071426  129372600
       2013-02-12  58.719791  66.842857  ...  68.501427  152263300
       2013-02-13  58.608109  66.715714  ...  66.744286  118801900
...                      ...        ...  ...        ...        ...
TWTR   2020-04-24  28.740000  28.740000  ...  27.990000   15936400
       2020-04-27  30.000000  30.000000  ...  29.129999   22490600
       2020-04-28  28.790001  28.790001  ...  30.400000   23364000
       2020-04-29  31.090000  31.090000  ...  30.059999   29568800
       2020-04-30  28.680000  28.680000  ...  28.790001   68791300
       2020-05-01  27.840000  27.840000  ...  28.000000   28231100
       2020-05-04  28.230000  28.230000  ...  27.600000   17846300
       2020-05-05  28.070000  28.070000  ...  28.670000   19221900
       2020-05-06  27.680000  27.680000  ...  28.350000   15531000
       2020-05-07  28.770000  28.770000  ...  28.080000   16190500
       2020-05-08  29.930000  29.930000  ...  29.090000   19208800
       2020-05-11  29.690001  29.690001  ...  29.650000   13831900
       2020-05-12  29.150000  29.150000  ...  29.850000   13734500
       2020-05-13  28.129999  28.129999  ...  29.240000   19401700
       2020-05-14  28.559999  28.559999  ...  27.830000   12364700
       2020-05-15  29.000000  29.000000  ...  28.299999   15939200
       2020-05-18  29.639999  29.639999  ...  29.700001   15790000
       2020-05-19  29.879999  29.879999  ...  29.700001   17386200
       2020-05-20  32.230000  32.230000  ...  30.500000   26955500
       2020-05-21  32.340000  32.340000  ...  32.099998   22881700
       2020-05-22  32.619999  32.619999  ...  32.380001   13310700
       2020-05-26  34.009998  34.009998  ...  33.599998   20489300
       2020-05-27  33.070000  33.070000  ...  33.630001   24550900
       2020-05-28  31.600000  31.600000  ...  31.500000   37254900
       2020-05-29  30.969999  30.969999  ...  31.559999   39427700
       2020-06-01  31.889999  31.889999  ...  31.059999   14583600
       2020-06-02  32.259998  32.259998  ...  32.189999   15318400

In [15]:
# Also only pulling the ticker, date and adj. close columns for our tickers.

adj_close = all_data[['Adj Close']].reset_index()
adj_close.head()

,Ticker,Date,Adj Close
0,AAPL,2013-01-02,68.502106
1,AAPL,2013-01-03,67.637436
2,AAPL,2013-01-04,65.753418
3,AAPL,2013-01-07,65.366646
4,AAPL,2013-01-08,65.542587


In [16]:
# Grabbing the ticker close from the end of last year
adj_close_start = adj_close[adj_close['Date']==end_of_last_year]
adj_close_start.head()

,Ticker,Date,Adj Close
1761,AAPL,2019-12-31,292.163818
3631,JNJ,2019-12-31,143.947876
5501,MCD,2019-12-31,195.080414
6644,MTCH,2019-12-31,82.110001
8514,NFLX,2019-12-31,323.570007


In [17]:
# Grab the latest stock close price

adj_close_latest = adj_close[adj_close['Date']==(stocks_end-datetime.timedelta(1))]
adj_close_latest

,Ticker,Date,Adj Close
1869,AAPL,2020-06-05,331.500000
3739,JNJ,2020-06-05,147.300003
5609,MCD,2020-06-05,197.160004
6752,MTCH,2020-06-05,90.089996
8622,NFLX,2020-06-05,419.600006
10492,WMT,2020-06-05,121.559998
12362,FB,2020-06-05,230.770004
14017,TWTR,2020-06-05,34.869999


In [18]:
adj_close_latest.set_index('Ticker', inplace=True)
adj_close_latest.head()

,Date,Adj Close
Ticker,,
AAPL,2020-06-05,331.500000
JNJ,2020-06-05,147.300003
MCD,2020-06-05,197.160004
MTCH,2020-06-05,90.089996
NFLX,2020-06-05,419.600006


In [19]:
portfolio_df.set_index(['Ticker'], inplace=True)

portfolio_df.head()

,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year
Ticker,,,,,
AAPL,2013-02-07,125,65.40,8175.00,2019-12-31
JNJ,2014-02-27,100,81.90,8190.00,2019-12-31
MCD,2013-06-04,100,84.99,8499.00,2019-12-31
MTCH,2015-12-14,600,13.63,8178.00,2019-12-31
NFLX,2016-01-14,75,108.71,8153.25,2019-12-31


In [20]:
# Merge the portfolio dataframe with the adj close dataframe; they are being joined by their indexes.

merged_portfolio = pd.merge(portfolio_df, adj_close_latest, left_index=True, right_index=True)
merged_portfolio

,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Date,Adj Close
Ticker,,,,,,,
AAPL,2013-02-07,125,65.40,8175.00,2019-12-31,2020-06-05,331.500000
JNJ,2014-02-27,100,81.90,8190.00,2019-12-31,2020-06-05,147.300003
MCD,2013-06-04,100,84.99,8499.00,2019-12-31,2020-06-05,197.160004
MTCH,2015-12-14,600,13.63,8178.00,2019-12-31,2020-06-05,90.089996
NFLX,2016-01-14,75,108.71,8153.25,2019-12-31,2020-06-05,419.600006
WMT,2013-08-14,125,68.30,8537.50,2019-12-31,2020-06-05,121.559998
FB,2013-12-13,150,53.32,7998.00,2019-12-31,2020-06-05,230.770004
TWTR,2015-01-05,225,36.38,8185.50,2019-12-31,2020-06-05,34.869999


In [21]:
# The below creates a new column which is the ticker return; takes the latest adjusted close for each position
# and divides that by the initial share cost.

merged_portfolio['ticker return'] = merged_portfolio['Adj Close'] / merged_portfolio['Unit Cost'] - 1

merged_portfolio

,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Date,Adj Close,ticker return
Ticker,,,,,,,,
AAPL,2013-02-07,125,65.40,8175.00,2019-12-31,2020-06-05,331.500000,4.068807
JNJ,2014-02-27,100,81.90,8190.00,2019-12-31,2020-06-05,147.300003,0.798535
MCD,2013-06-04,100,84.99,8499.00,2019-12-31,2020-06-05,197.160004,1.319802
MTCH,2015-12-14,600,13.63,8178.00,2019-12-31,2020-06-05,90.089996,5.609684
NFLX,2016-01-14,75,108.71,8153.25,2019-12-31,2020-06-05,419.600006,2.859811
WMT,2013-08-14,125,68.30,8537.50,2019-12-31,2020-06-05,121.559998,0.779795
FB,2013-12-13,150,53.32,7998.00,2019-12-31,2020-06-05,230.770004,3.328020
TWTR,2015-01-05,225,36.38,8185.50,2019-12-31,2020-06-05,34.869999,-0.041506


In [22]:
merged_portfolio.reset_index(inplace=True)

In [23]:
type(merged_portfolio['Acquisition Date'][1])

pandas._libs.tslibs.timestamps.Timestamp

In [24]:
# Here we are merging the new dataframe with the sp500 adjusted closes since the sp start price based on 
# each ticker's acquisition date and sp500 close date.

merged_portfolio_sp = pd.merge(merged_portfolio, sp_500_adj_close, left_on='Acquisition Date', right_on='Date')
# .set_index('Ticker')

merged_portfolio_sp.head()

,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Date_x,Adj Close_x,ticker return,Date_y,Adj Close_y
0,AAPL,2013-02-07,125,65.40,8175.00,2019-12-31,2020-06-05,331.500000,4.068807,2013-02-07,1509.390015
1,JNJ,2014-02-27,100,81.90,8190.00,2019-12-31,2020-06-05,147.300003,0.798535,2014-02-27,1854.290039
2,MCD,2013-06-04,100,84.99,8499.00,2019-12-31,2020-06-05,197.160004,1.319802,2013-06-04,1631.380005
3,MTCH,2015-12-14,600,13.63,8178.00,2019-12-31,2020-06-05,90.089996,5.609684,2015-12-14,2021.939941
4,NFLX,2016-01-14,75,108.71,8153.25,2019-12-31,2020-06-05,419.600006,2.859811,2016-01-14,1921.839966


In [25]:
# We will delete the additional date column which is created from this merge.
# We then rename columns to Latest Date and then reflect Ticker Adj Close and SP 500 Initial Close.

del merged_portfolio_sp['Date_y']

merged_portfolio_sp.rename(columns={'Date_x': 'Latest Date', 'Adj Close_x': 'Ticker Adj Close'
                                    , 'Adj Close_y': 'SP 500 Initial Close'}, inplace=True)

merged_portfolio_sp.head()

,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close
0,AAPL,2013-02-07,125,65.40,8175.00,2019-12-31,2020-06-05,331.500000,4.068807,1509.390015
1,JNJ,2014-02-27,100,81.90,8190.00,2019-12-31,2020-06-05,147.300003,0.798535,1854.290039
2,MCD,2013-06-04,100,84.99,8499.00,2019-12-31,2020-06-05,197.160004,1.319802,1631.380005
3,MTCH,2015-12-14,600,13.63,8178.00,2019-12-31,2020-06-05,90.089996,5.609684,2021.939941
4,NFLX,2016-01-14,75,108.71,8153.25,2019-12-31,2020-06-05,419.600006,2.859811,1921.839966


In [26]:
# This new column determines what SP 500 equivalent purchase would have been at purchase date of stock.
merged_portfolio_sp['Equiv SP Shares'] = merged_portfolio_sp['Cost Basis'] / merged_portfolio_sp['SP 500 Initial Close']
merged_portfolio_sp

,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close,Equiv SP Shares
0,AAPL,2013-02-07,125,65.40,8175.00,2019-12-31,2020-06-05,331.500000,4.068807,1509.390015,5.416095
1,JNJ,2014-02-27,100,81.90,8190.00,2019-12-31,2020-06-05,147.300003,0.798535,1854.290039,4.416785
2,MCD,2013-06-04,100,84.99,8499.00,2019-12-31,2020-06-05,197.160004,1.319802,1631.380005,5.209700
3,MTCH,2015-12-14,600,13.63,8178.00,2019-12-31,2020-06-05,90.089996,5.609684,2021.939941,4.044631
4,NFLX,2016-01-14,75,108.71,8153.25,2019-12-31,2020-06-05,419.600006,2.859811,1921.839966,4.242419
5,WMT,2013-08-14,125,68.30,8537.50,2019-12-31,2020-06-05,121.559998,0.779795,1685.390015,5.065593
6,FB,2013-12-13,150,53.32,7998.00,2019-12-31,2020-06-05,230.770004,3.328020,1775.319946,4.505103
7,TWTR,2015-01-05,225,36.38,8185.50,2019-12-31,2020-06-05,34.869999,-0.041506,2020.579956,4.051065


In [27]:
sp_500_adj_close.head()

,Date,Adj Close
0,2013-01-02,1462.420044
1,2013-01-03,1459.369995
2,2013-01-04,1466.469971
3,2013-01-07,1461.890015
4,2013-01-08,1457.150024


In [28]:
# We are joining the developing dataframe with the sp500 closes again, this time with the latest close for SP.
merged_portfolio_sp_latest = pd.merge(merged_portfolio_sp, sp_500_adj_close, left_on='Latest Date', right_on='Date')

merged_portfolio_sp_latest.head()

,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close,Equiv SP Shares,Date,Adj Close
0,AAPL,2013-02-07,125,65.40,8175.00,2019-12-31,2020-06-05,331.500000,4.068807,1509.390015,5.416095,2020-06-05,3193.929932
1,JNJ,2014-02-27,100,81.90,8190.00,2019-12-31,2020-06-05,147.300003,0.798535,1854.290039,4.416785,2020-06-05,3193.929932
2,MCD,2013-06-04,100,84.99,8499.00,2019-12-31,2020-06-05,197.160004,1.319802,1631.380005,5.209700,2020-06-05,3193.929932
3,MTCH,2015-12-14,600,13.63,8178.00,2019-12-31,2020-06-05,90.089996,5.609684,2021.939941,4.044631,2020-06-05,3193.929932
4,NFLX,2016-01-14,75,108.71,8153.25,2019-12-31,2020-06-05,419.600006,2.859811,1921.839966,4.242419,2020-06-05,3193.929932


In [29]:
# Once again need to delete the new Date column added as it's redundant to Latest Date.  
# Modify Adj Close from the sp dataframe to distinguish it by calling it the SP 500 Latest Close.

del merged_portfolio_sp_latest['Date']

merged_portfolio_sp_latest.rename(columns={'Adj Close': 'SP 500 Latest Close'}, inplace=True)

merged_portfolio_sp_latest.head()

,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close,Equiv SP Shares,SP 500 Latest Close
0,AAPL,2013-02-07,125,65.40,8175.00,2019-12-31,2020-06-05,331.500000,4.068807,1509.390015,5.416095,3193.929932
1,JNJ,2014-02-27,100,81.90,8190.00,2019-12-31,2020-06-05,147.300003,0.798535,1854.290039,4.416785,3193.929932
2,MCD,2013-06-04,100,84.99,8499.00,2019-12-31,2020-06-05,197.160004,1.319802,1631.380005,5.209700,3193.929932
3,MTCH,2015-12-14,600,13.63,8178.00,2019-12-31,2020-06-05,90.089996,5.609684,2021.939941,4.044631,3193.929932
4,NFLX,2016-01-14,75,108.71,8153.25,2019-12-31,2020-06-05,419.600006,2.859811,1921.839966,4.242419,3193.929932


In [30]:
# Percent return of SP from acquisition date of position through latest trading day.
merged_portfolio_sp_latest['SP Return'] = merged_portfolio_sp_latest['SP 500 Latest Close'] / merged_portfolio_sp_latest['SP 500 Initial Close'] - 1

# This is a new column which takes the tickers return and subtracts the sp 500 equivalent range return.
merged_portfolio_sp_latest['Abs. Return Compare'] = merged_portfolio_sp_latest['ticker return'] - merged_portfolio_sp_latest['SP Return']

# This is a new column where we calculate the ticker's share value by multiplying the original quantity by the latest close.
merged_portfolio_sp_latest['Ticker Share Value'] = merged_portfolio_sp_latest['Quantity'] * merged_portfolio_sp_latest['Ticker Adj Close']

# We calculate the equivalent SP 500 Value if we take the original SP shares * the latest SP 500 share price.
merged_portfolio_sp_latest['SP 500 Value'] = merged_portfolio_sp_latest['Equiv SP Shares'] * merged_portfolio_sp_latest['SP 500 Latest Close']

# This is a new column where we take the current market value for the shares and subtract the SP 500 value.
merged_portfolio_sp_latest['Abs Value Compare'] = merged_portfolio_sp_latest['Ticker Share Value'] - merged_portfolio_sp_latest['SP 500 Value']

# This column calculates profit / loss for stock position.
merged_portfolio_sp_latest['Stock Gain / (Loss)'] = merged_portfolio_sp_latest['Ticker Share Value'] - merged_portfolio_sp_latest['Cost Basis']

# This column calculates profit / loss for SP 500.
merged_portfolio_sp_latest['SP 500 Gain / (Loss)'] = merged_portfolio_sp_latest['SP 500 Value'] - merged_portfolio_sp_latest['Cost Basis']

merged_portfolio_sp_latest.head()

,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close,Equiv SP Shares,SP 500 Latest Close,SP Return,Abs. Return Compare,Ticker Share Value,SP 500 Value,Abs Value Compare,Stock Gain / (Loss),SP 500 Gain / (Loss)
0,AAPL,2013-02-07,125,65.40,8175.00,2019-12-31,2020-06-05,331.500000,4.068807,1509.390015,5.416095,3193.929932,1.116040,2.952767,41437.500000,17298.628544,24138.871456,33262.500000,9123.628544
1,JNJ,2014-02-27,100,81.90,8190.00,2019-12-31,2020-06-05,147.300003,0.798535,1854.290039,4.416785,3193.929932,0.722454,0.076080,14730.000305,14106.901072,623.099233,6540.000305,5916.901072
2,MCD,2013-06-04,100,84.99,8499.00,2019-12-31,2020-06-05,197.160004,1.319802,1631.380005,5.209700,3193.929932,0.957809,0.361994,19716.000366,16639.415959,3076.584407,11217.000366,8140.415959
3,MTCH,2015-12-14,600,13.63,8178.00,2019-12-31,2020-06-05,90.089996,5.609684,2021.939941,4.044631,3193.929932,0.579636,5.030048,54053.997803,12918.266486,41135.731317,45875.997803,4740.266486
4,NFLX,2016-01-14,75,108.71,8153.25,2019-12-31,2020-06-05,419.600006,2.859811,1921.839966,4.242419,3193.929932,0.661913,2.197898,31470.000458,13549.988385,17920.012073,23316.750458,5396.738385


In [31]:
# Merge the overall dataframe with the adj close start of year dataframe for YTD tracking of tickers.
# Should not need to do the outer join;

merged_portfolio_sp_latest_YTD = pd.merge(merged_portfolio_sp_latest, adj_close_start, on='Ticker')
# , how='outer'

merged_portfolio_sp_latest_YTD.head()

,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close,Equiv SP Shares,SP 500 Latest Close,SP Return,Abs. Return Compare,Ticker Share Value,SP 500 Value,Abs Value Compare,Stock Gain / (Loss),SP 500 Gain / (Loss),Date,Adj Close
0,AAPL,2013-02-07,125,65.40,8175.00,2019-12-31,2020-06-05,331.500000,4.068807,1509.390015,5.416095,3193.929932,1.116040,2.952767,41437.500000,17298.628544,24138.871456,33262.500000,9123.628544,2019-12-31,292.163818
1,JNJ,2014-02-27,100,81.90,8190.00,2019-12-31,2020-06-05,147.300003,0.798535,1854.290039,4.416785,3193.929932,0.722454,0.076080,14730.000305,14106.901072,623.099233,6540.000305,5916.901072,2019-12-31,143.947876
2,MCD,2013-06-04,100,84.99,8499.00,2019-12-31,2020-06-05,197.160004,1.319802,1631.380005,5.209700,3193.929932,0.957809,0.361994,19716.000366,16639.415959,3076.584407,11217.000366,8140.415959,2019-12-31,195.080414
3,MTCH,2015-12-14,600,13.63,8178.00,2019-12-31,2020-06-05,90.089996,5.609684,2021.939941,4.044631,3193.929932,0.579636,5.030048,54053.997803,12918.266486,41135.731317,45875.997803,4740.266486,2019-12-31,82.110001
4,NFLX,2016-01-14,75,108.71,8153.25,2019-12-31,2020-06-05,419.600006,2.859811,1921.839966,4.242419,3193.929932,0.661913,2.197898,31470.000458,13549.988385,17920.012073,23316.750458,5396.738385,2019-12-31,323.570007


In [32]:
# Deleting date again as it's an unnecessary column.  Explaining that new column is the Ticker Start of Year Close.

del merged_portfolio_sp_latest_YTD['Date']

merged_portfolio_sp_latest_YTD.rename(columns={'Adj Close': 'Ticker Start Year Close'}, inplace=True)

merged_portfolio_sp_latest_YTD.head()

,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close,Equiv SP Shares,SP 500 Latest Close,SP Return,Abs. Return Compare,Ticker Share Value,SP 500 Value,Abs Value Compare,Stock Gain / (Loss),SP 500 Gain / (Loss),Ticker Start Year Close
0,AAPL,2013-02-07,125,65.40,8175.00,2019-12-31,2020-06-05,331.500000,4.068807,1509.390015,5.416095,3193.929932,1.116040,2.952767,41437.500000,17298.628544,24138.871456,33262.500000,9123.628544,292.163818
1,JNJ,2014-02-27,100,81.90,8190.00,2019-12-31,2020-06-05,147.300003,0.798535,1854.290039,4.416785,3193.929932,0.722454,0.076080,14730.000305,14106.901072,623.099233,6540.000305,5916.901072,143.947876
2,MCD,2013-06-04,100,84.99,8499.00,2019-12-31,2020-06-05,197.160004,1.319802,1631.380005,5.209700,3193.929932,0.957809,0.361994,19716.000366,16639.415959,3076.584407,11217.000366,8140.415959,195.080414
3,MTCH,2015-12-14,600,13.63,8178.00,2019-12-31,2020-06-05,90.089996,5.609684,2021.939941,4.044631,3193.929932,0.579636,5.030048,54053.997803,12918.266486,41135.731317,45875.997803,4740.266486,82.110001
4,NFLX,2016-01-14,75,108.71,8153.25,2019-12-31,2020-06-05,419.600006,2.859811,1921.839966,4.242419,3193.929932,0.661913,2.197898,31470.000458,13549.988385,17920.012073,23316.750458,5396.738385,323.570007


In [33]:
sp_500_adj_close_start
merged_portfolio_sp_latest_YTD

,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close,Equiv SP Shares,SP 500 Latest Close,SP Return,Abs. Return Compare,Ticker Share Value,SP 500 Value,Abs Value Compare,Stock Gain / (Loss),SP 500 Gain / (Loss),Ticker Start Year Close
0,AAPL,2013-02-07,125,65.40,8175.00,2019-12-31,2020-06-05,331.500000,4.068807,1509.390015,5.416095,3193.929932,1.116040,2.952767,41437.500000,17298.628544,24138.871456,33262.500000,9123.628544,292.163818
1,JNJ,2014-02-27,100,81.90,8190.00,2019-12-31,2020-06-05,147.300003,0.798535,1854.290039,4.416785,3193.929932,0.722454,0.076080,14730.000305,14106.901072,623.099233,6540.000305,5916.901072,143.947876
2,MCD,2013-06-04,100,84.99,8499.00,2019-12-31,2020-06-05,197.160004,1.319802,1631.380005,5.209700,3193.929932,0.957809,0.361994,19716.000366,16639.415959,3076.584407,11217.000366,8140.415959,195.080414
3,MTCH,2015-12-14,600,13.63,8178.00,2019-12-31,2020-06-05,90.089996,5.609684,2021.939941,4.044631,3193.929932,0.579636,5.030048,54053.997803,12918.266486,41135.731317,45875.997803,4740.266486,82.110001
4,NFLX,2016-01-14,75,108.71,8153.25,2019-12-31,2020-06-05,419.600006,2.859811,1921.839966,4.242419,3193.929932,0.661913,2.197898,31470.000458,13549.988385,17920.012073,23316.750458,5396.738385,323.570007
5,WMT,2013-08-14,125,68.30,8537.50,2019-12-31,2020-06-05,121.559998,0.779795,1685.390015,5.065593,3193.929932,0.895069,-0.115274,15194.999695,16179.149369,-984.149674,6657.499695,7641.649369,117.798302
6,FB,2013-12-13,150,53.32,7998.00,2019-12-31,2020-06-05,230.770004,3.328020,1775.319946,4.505103,3193.929932,0.799073,2.528947,34615.500641,14388.984727,20226.515914,26617.500641,6390.984727,205.250000
7,TWTR,2015-01-05,225,36.38,8185.50,2019-12-31,2020-06-05,34.869999,-0.041506,2020.579956,4.051065,3193.929932,0.580700,-0.622206,7845.749760,12938.816589,-5093.066830,-339.750240,4753.316589,32.049999


In [34]:
# Join the SP 500 start of year with current dataframe for SP 500 ytd comparisons to tickers.

merged_portfolio_sp_latest_YTD_sp = pd.merge(merged_portfolio_sp_latest_YTD, sp_500_adj_close_start, left_on='Start of Year', right_on='Date')

merged_portfolio_sp_latest_YTD_sp.head()

,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close,Equiv SP Shares,SP 500 Latest Close,SP Return,Abs. Return Compare,Ticker Share Value,SP 500 Value,Abs Value Compare,Stock Gain / (Loss),SP 500 Gain / (Loss),Ticker Start Year Close,Date,Adj Close
0,AAPL,2013-02-07,125,65.40,8175.00,2019-12-31,2020-06-05,331.500000,4.068807,1509.390015,5.416095,3193.929932,1.116040,2.952767,41437.500000,17298.628544,24138.871456,33262.500000,9123.628544,292.163818,2019-12-31,3230.780029
1,JNJ,2014-02-27,100,81.90,8190.00,2019-12-31,2020-06-05,147.300003,0.798535,1854.290039,4.416785,3193.929932,0.722454,0.076080,14730.000305,14106.901072,623.099233,6540.000305,5916.901072,143.947876,2019-12-31,3230.780029
2,MCD,2013-06-04,100,84.99,8499.00,2019-12-31,2020-06-05,197.160004,1.319802,1631.380005,5.209700,3193.929932,0.957809,0.361994,19716.000366,16639.415959,3076.584407,11217.000366,8140.415959,195.080414,2019-12-31,3230.780029
3,MTCH,2015-12-14,600,13.63,8178.00,2019-12-31,2020-06-05,90.089996,5.609684,2021.939941,4.044631,3193.929932,0.579636,5.030048,54053.997803,12918.266486,41135.731317,45875.997803,4740.266486,82.110001,2019-12-31,3230.780029
4,NFLX,2016-01-14,75,108.71,8153.25,2019-12-31,2020-06-05,419.600006,2.859811,1921.839966,4.242419,3193.929932,0.661913,2.197898,31470.000458,13549.988385,17920.012073,23316.750458,5396.738385,323.570007,2019-12-31,3230.780029


In [35]:
# Deleting another unneeded Date column.

del merged_portfolio_sp_latest_YTD_sp['Date']

# Renaming so that it's clear this column is SP 500 start of year close.
merged_portfolio_sp_latest_YTD_sp.rename(columns={'Adj Close': 'SP Start Year Close'}, inplace=True)

# YTD return for portfolio position.
merged_portfolio_sp_latest_YTD_sp['Share YTD'] = merged_portfolio_sp_latest_YTD_sp['Ticker Adj Close'] / merged_portfolio_sp_latest_YTD_sp['Ticker Start Year Close'] - 1

# YTD return for SP to run compares.
merged_portfolio_sp_latest_YTD_sp['SP 500 YTD'] = merged_portfolio_sp_latest_YTD_sp['SP 500 Latest Close'] / merged_portfolio_sp_latest_YTD_sp['SP Start Year Close'] - 1

merged_portfolio_sp_latest_YTD_sp.head()

,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close,Equiv SP Shares,SP 500 Latest Close,SP Return,Abs. Return Compare,Ticker Share Value,SP 500 Value,Abs Value Compare,Stock Gain / (Loss),SP 500 Gain / (Loss),Ticker Start Year Close,SP Start Year Close,Share YTD,SP 500 YTD
0,AAPL,2013-02-07,125,65.40,8175.00,2019-12-31,2020-06-05,331.500000,4.068807,1509.390015,5.416095,3193.929932,1.116040,2.952767,41437.500000,17298.628544,24138.871456,33262.500000,9123.628544,292.163818,3230.780029,0.134637,-0.011406
1,JNJ,2014-02-27,100,81.90,8190.00,2019-12-31,2020-06-05,147.300003,0.798535,1854.290039,4.416785,3193.929932,0.722454,0.076080,14730.000305,14106.901072,623.099233,6540.000305,5916.901072,143.947876,3230.780029,0.023287,-0.011406
2,MCD,2013-06-04,100,84.99,8499.00,2019-12-31,2020-06-05,197.160004,1.319802,1631.380005,5.209700,3193.929932,0.957809,0.361994,19716.000366,16639.415959,3076.584407,11217.000366,8140.415959,195.080414,3230.780029,0.010660,-0.011406
3,MTCH,2015-12-14,600,13.63,8178.00,2019-12-31,2020-06-05,90.089996,5.609684,2021.939941,4.044631,3193.929932,0.579636,5.030048,54053.997803,12918.266486,41135.731317,45875.997803,4740.266486,82.110001,3230.780029,0.097187,-0.011406
4,NFLX,2016-01-14,75,108.71,8153.25,2019-12-31,2020-06-05,419.600006,2.859811,1921.839966,4.242419,3193.929932,0.661913,2.197898,31470.000458,13549.988385,17920.012073,23316.750458,5396.738385,323.570007,3230.780029,0.296783,-0.011406


In [36]:
merged_portfolio_sp_latest_YTD_sp = merged_portfolio_sp_latest_YTD_sp.sort_values(by='Ticker', ascending=True)
merged_portfolio_sp_latest_YTD_sp

,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close,Equiv SP Shares,SP 500 Latest Close,SP Return,Abs. Return Compare,Ticker Share Value,SP 500 Value,Abs Value Compare,Stock Gain / (Loss),SP 500 Gain / (Loss),Ticker Start Year Close,SP Start Year Close,Share YTD,SP 500 YTD
0,AAPL,2013-02-07,125,65.40,8175.00,2019-12-31,2020-06-05,331.500000,4.068807,1509.390015,5.416095,3193.929932,1.116040,2.952767,41437.500000,17298.628544,24138.871456,33262.500000,9123.628544,292.163818,3230.780029,0.134637,-0.011406
6,FB,2013-12-13,150,53.32,7998.00,2019-12-31,2020-06-05,230.770004,3.328020,1775.319946,4.505103,3193.929932,0.799073,2.528947,34615.500641,14388.984727,20226.515914,26617.500641,6390.984727,205.250000,3230.780029,0.124336,-0.011406
1,JNJ,2014-02-27,100,81.90,8190.00,2019-12-31,2020-06-05,147.300003,0.798535,1854.290039,4.416785,3193.929932,0.722454,0.076080,14730.000305,14106.901072,623.099233,6540.000305,5916.901072,143.947876,3230.780029,0.023287,-0.011406
2,MCD,2013-06-04,100,84.99,8499.00,2019-12-31,2020-06-05,197.160004,1.319802,1631.380005,5.209700,3193.929932,0.957809,0.361994,19716.000366,16639.415959,3076.584407,11217.000366,8140.415959,195.080414,3230.780029,0.010660,-0.011406
3,MTCH,2015-12-14,600,13.63,8178.00,2019-12-31,2020-06-05,90.089996,5.609684,2021.939941,4.044631,3193.929932,0.579636,5.030048,54053.997803,12918.266486,41135.731317,45875.997803,4740.266486,82.110001,3230.780029,0.097187,-0.011406
4,NFLX,2016-01-14,75,108.71,8153.25,2019-12-31,2020-06-05,419.600006,2.859811,1921.839966,4.242419,3193.929932,0.661913,2.197898,31470.000458,13549.988385,17920.012073,23316.750458,5396.738385,323.570007,3230.780029,0.296783,-0.011406
7,TWTR,2015-01-05,225,36.38,8185.50,2019-12-31,2020-06-05,34.869999,-0.041506,2020.579956,4.051065,3193.929932,0.580700,-0.622206,7845.749760,12938.816589,-5093.066830,-339.750240,4753.316589,32.049999,3230.780029,0.087988,-0.011406
5,WMT,2013-08-14,125,68.30,8537.50,2019-12-31,2020-06-05,121.559998,0.779795,1685.390015,5.065593,3193.929932,0.895069,-0.115274,15194.999695,16179.149369,-984.149674,6657.499695,7641.649369,117.798302,3230.780029,0.031933,-0.011406


In [37]:
# Cumulative sum of original investment
merged_portfolio_sp_latest_YTD_sp['Cum Invst'] = merged_portfolio_sp_latest_YTD_sp['Cost Basis'].cumsum()

# Cumulative sum of Ticker Share Value (latest FMV based on initial quantity purchased).
merged_portfolio_sp_latest_YTD_sp['Cum Ticker Returns'] = merged_portfolio_sp_latest_YTD_sp['Ticker Share Value'].cumsum()

# Cumulative sum of SP Share Value (latest FMV driven off of initial SP equiv purchase).
merged_portfolio_sp_latest_YTD_sp['Cum SP Returns'] = merged_portfolio_sp_latest_YTD_sp['SP 500 Value'].cumsum()

# Cumulative CoC multiple return for stock investments
merged_portfolio_sp_latest_YTD_sp['Cum Ticker ROI Mult'] = merged_portfolio_sp_latest_YTD_sp['Cum Ticker Returns'] / merged_portfolio_sp_latest_YTD_sp['Cum Invst']

merged_portfolio_sp_latest_YTD_sp.head()


,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close,Equiv SP Shares,SP 500 Latest Close,SP Return,Abs. Return Compare,Ticker Share Value,SP 500 Value,Abs Value Compare,Stock Gain / (Loss),SP 500 Gain / (Loss),Ticker Start Year Close,SP Start Year Close,Share YTD,SP 500 YTD,Cum Invst,Cum Ticker Returns,Cum SP Returns,Cum Ticker ROI Mult
0,AAPL,2013-02-07,125,65.40,8175.0,2019-12-31,2020-06-05,331.500000,4.068807,1509.390015,5.416095,3193.929932,1.116040,2.952767,41437.500000,17298.628544,24138.871456,33262.500000,9123.628544,292.163818,3230.780029,0.134637,-0.011406,8175.0,41437.500000,17298.628544,5.068807
6,FB,2013-12-13,150,53.32,7998.0,2019-12-31,2020-06-05,230.770004,3.328020,1775.319946,4.505103,3193.929932,0.799073,2.528947,34615.500641,14388.984727,20226.515914,26617.500641,6390.984727,205.250000,3230.780029,0.124336,-0.011406,16173.0,76053.000641,31687.613271,4.702467
1,JNJ,2014-02-27,100,81.90,8190.0,2019-12-31,2020-06-05,147.300003,0.798535,1854.290039,4.416785,3193.929932,0.722454,0.076080,14730.000305,14106.901072,623.099233,6540.000305,5916.901072,143.947876,3230.780029,0.023287,-0.011406,24363.0,90783.000946,45794.514343,3.726265
2,MCD,2013-06-04,100,84.99,8499.0,2019-12-31,2020-06-05,197.160004,1.319802,1631.380005,5.209700,3193.929932,0.957809,0.361994,19716.000366,16639.415959,3076.584407,11217.000366,8140.415959,195.080414,3230.780029,0.010660,-0.011406,32862.0,110499.001312,62433.930303,3.362516
3,MTCH,2015-12-14,600,13.63,8178.0,2019-12-31,2020-06-05,90.089996,5.609684,2021.939941,4.044631,3193.929932,0.579636,5.030048,54053.997803,12918.266486,41135.731317,45875.997803,4740.266486,82.110001,3230.780029,0.097187,-0.011406,41040.0,164552.999115,75352.196788,4.009576


## Assessing Where Positions are At versus Highest Close

In [38]:
# Referencing the adj_close dataframe from above

adj_close.head()

,Ticker,Date,Adj Close
0,AAPL,2013-01-02,68.502106
1,AAPL,2013-01-03,67.637436
2,AAPL,2013-01-04,65.753418
3,AAPL,2013-01-07,65.366646
4,AAPL,2013-01-08,65.542587


In [39]:
portfolio_df.head()

,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year
Ticker,,,,,
AAPL,2013-02-07,125,65.40,8175.00,2019-12-31
JNJ,2014-02-27,100,81.90,8190.00,2019-12-31
MCD,2013-06-04,100,84.99,8499.00,2019-12-31
MTCH,2015-12-14,600,13.63,8178.00,2019-12-31
NFLX,2016-01-14,75,108.71,8153.25,2019-12-31


In [40]:
# Need to factor in that some positions were purchased much more recently than others.
# Join adj_close dataframe with portfolio in order to have acquisition date.

portfolio_df.reset_index(inplace=True)

adj_close_acq_date = pd.merge(adj_close, portfolio_df, on='Ticker')

adj_close_acq_date.head()

,Ticker,Date,Adj Close,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year
0,AAPL,2013-01-02,68.502106,2013-02-07,125,65.4,8175.0,2019-12-31
1,AAPL,2013-01-03,67.637436,2013-02-07,125,65.4,8175.0,2019-12-31
2,AAPL,2013-01-04,65.753418,2013-02-07,125,65.4,8175.0,2019-12-31
3,AAPL,2013-01-07,65.366646,2013-02-07,125,65.4,8175.0,2019-12-31
4,AAPL,2013-01-08,65.542587,2013-02-07,125,65.4,8175.0,2019-12-31


In [41]:
# delete_columns = ['Quantity', 'Unit Cost', 'Cost Basis', 'Start of Year']

del adj_close_acq_date['Quantity']
del adj_close_acq_date['Unit Cost']
del adj_close_acq_date['Cost Basis']
del adj_close_acq_date['Start of Year']

# Sort by these columns in this order in order to make it clearer where compare for each position should begin.
adj_close_acq_date.sort_values(by=['Ticker', 'Acquisition Date', 'Date'], ascending=[True, True, True], inplace=True)

In [42]:
# Anything less than 0 means that the stock close was prior to acquisition.

adj_close_acq_date['Date Delta'] = pd.to_datetime(adj_close_acq_date['Date']) - pd.to_datetime(adj_close_acq_date['Acquisition Date'])

adj_close_acq_date['Date Delta'] = adj_close_acq_date[['Date Delta']].apply(pd.to_numeric)  

adj_close_acq_date.head()




,Ticker,Date,Adj Close,Acquisition Date,Date Delta
0,AAPL,2013-01-02,68.502106,2013-02-07,-3110400000000000
1,AAPL,2013-01-03,67.637436,2013-02-07,-3024000000000000
2,AAPL,2013-01-04,65.753418,2013-02-07,-2937600000000000
3,AAPL,2013-01-07,65.366646,2013-02-07,-2678400000000000
4,AAPL,2013-01-08,65.542587,2013-02-07,-2592000000000000


In [43]:
# Modified the dataframe being evaluated to look at highest close which occurred after Acquisition Date (aka, not prior to purchase).

adj_close_acq_date_modified = adj_close_acq_date[adj_close_acq_date['Date Delta']>=0]

adj_close_acq_date_modified.head()

,Ticker,Date,Adj Close,Acquisition Date,Date Delta
25,AAPL,2013-02-07,58.759972,2013-02-07,0
26,AAPL,2013-02-08,59.608299,2013-02-07,86400000000000
27,AAPL,2013-02-11,60.229511,2013-02-07,345600000000000
28,AAPL,2013-02-12,58.719791,2013-02-07,432000000000000
29,AAPL,2013-02-13,58.608109,2013-02-07,518400000000000


In [44]:
# This pivot table will index on the Ticker and Acquisition Date, and find the max adjusted close.

adj_close_pivot = adj_close_acq_date_modified.pivot_table(index=['Ticker', 'Acquisition Date'], values='Adj Close', aggfunc=np.max)

adj_close_pivot.reset_index(inplace=True)

adj_close_pivot

,Ticker,Acquisition Date,Adj Close
0,AAPL,2013-02-07,331.500000
1,FB,2013-12-13,234.910004
2,JNJ,2014-02-27,154.439407
3,MCD,2013-06-04,215.830231
4,MTCH,2015-12-14,94.360001
5,NFLX,2016-01-14,454.190002
6,TWTR,2015-01-05,52.869999
7,WMT,2013-08-14,131.750458


In [45]:
# Merge the adj close pivot table with the adj_close table in order to grab the date of the Adj Close High (good to know).

adj_close_pivot_merged = pd.merge(adj_close_pivot, adj_close
                                             , on=['Ticker', 'Adj Close'])

adj_close_pivot_merged.head()

,Ticker,Acquisition Date,Adj Close,Date
0,AAPL,2013-02-07,331.500000,2020-06-05
1,FB,2013-12-13,234.910004,2020-05-22
2,JNJ,2014-02-27,154.439407,2020-04-23
3,MCD,2013-06-04,215.830231,2019-09-06
4,MTCH,2015-12-14,94.360001,2020-06-02


In [46]:
# Merge the Adj Close pivot table with the master dataframe to have the closing high since you have owned the stock.

merged_portfolio_sp_latest_YTD_sp_closing_high = pd.merge(merged_portfolio_sp_latest_YTD_sp, adj_close_pivot_merged
                                             , on=['Ticker', 'Acquisition Date'])

# Renaming so that it's clear that the new columns are two year closing high and two year closing high date.
merged_portfolio_sp_latest_YTD_sp_closing_high.rename(columns={'Adj Close': 'Closing High Adj Close', 'Date': 'Closing High Adj Close Date'}, inplace=True)

merged_portfolio_sp_latest_YTD_sp_closing_high['Pct off High'] = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker Adj Close'] / merged_portfolio_sp_latest_YTD_sp_closing_high['Closing High Adj Close'] - 1 

merged_portfolio_sp_latest_YTD_sp_closing_high

,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close,Equiv SP Shares,SP 500 Latest Close,SP Return,Abs. Return Compare,Ticker Share Value,SP 500 Value,Abs Value Compare,Stock Gain / (Loss),SP 500 Gain / (Loss),Ticker Start Year Close,SP Start Year Close,Share YTD,SP 500 YTD,Cum Invst,Cum Ticker Returns,Cum SP Returns,Cum Ticker ROI Mult,Closing High Adj Close,Closing High Adj Close Date,Pct off High
0,AAPL,2013-02-07,125,65.40,8175.00,2019-12-31,2020-06-05,331.500000,4.068807,1509.390015,5.416095,3193.929932,1.116040,2.952767,41437.500000,17298.628544,24138.871456,33262.500000,9123.628544,292.163818,3230.780029,0.134637,-0.011406,8175.00,41437.500000,17298.628544,5.068807,331.500000,2020-06-05,0.000000
1,FB,2013-12-13,150,53.32,7998.00,2019-12-31,2020-06-05,230.770004,3.328020,1775.319946,4.505103,3193.929932,0.799073,2.528947,34615.500641,14388.984727,20226.515914,26617.500641,6390.984727,205.250000,3230.780029,0.124336,-0.011406,16173.00,76053.000641,31687.613271,4.702467,234.910004,2020-05-22,-0.017624
2,JNJ,2014-02-27,100,81.90,8190.00,2019-12-31,2020-06-05,147.300003,0.798535,1854.290039,4.416785,3193.929932,0.722454,0.076080,14730.000305,14106.901072,623.099233,6540.000305,5916.901072,143.947876,3230.780029,0.023287,-0.011406,24363.00,90783.000946,45794.514343,3.726265,154.439407,2020-04-23,-0.046228
3,MCD,2013-06-04,100,84.99,8499.00,2019-12-31,2020-06-05,197.160004,1.319802,1631.380005,5.209700,3193.929932,0.957809,0.361994,19716.000366,16639.415959,3076.584407,11217.000366,8140.415959,195.080414,3230.780029,0.010660,-0.011406,32862.00,110499.001312,62433.930303,3.362516,215.830231,2019-09-06,-0.086504
4,MTCH,2015-12-14,600,13.63,8178.00,2019-12-31,2020-06-05,90.089996,5.609684,2021.939941,4.044631,3193.929932,0.579636,5.030048,54053.997803,12918.266486,41135.731317,45875.997803,4740.266486,82.110001,3230.780029,0.097187,-0.011406,41040.00,164552.999115,75352.196788,4.009576,94.360001,2020-06-02,-0.045252
5,NFLX,2016-01-14,75,108.71,8153.25,2019-12-31,2020-06-05,419.600006,2.859811,1921.839966,4.242419,3193.929932,0.661913,2.197898,31470.000458,13549.988385,17920.012073,23316.750458,5396.738385,323.570007,3230.780029,0.296783,-0.011406,49193.25,196022.999573,88902.185173,3.984754,454.190002,2020-05-15,-0.076158
6,TWTR,2015-01-05,225,36.38,8185.50,2019-12-31,2020-06-05,34.869999,-0.041506,2020.579956,4.051065,3193.929932,0.580700,-0.622206,7845.749760,12938.816589,-5093.066830,-339.750240,4753.316589,32.049999,3230.780029,0.087988,-0.011406,57378.75,203868.749332,101841.001763,3.553036,52.869999,2015-04-07,-0.340458
7,WMT,2013-08-14,125,68.30,8537.50,2019-12-31,2020-06-05,121.559998,0.779795,1685.390015,5.065593,3193.929932,0.895069,-0.115274,15194.999695,16179.149369,-984.149674,6657.499695,7641.649369,117.798302,3230.780029,0.031933,-0.011406,65916.25,219063.749027,118020.151131,3.323365,131.750458,2020-04-16,-0.077347


In [47]:
# Not needed for this blog post -- this is if you have multiple positions for the same ticker with different acquisition dates.
# merged_portfolio_sp_latest_YTD_sp_closing_high['Counts'] = merged_portfolio_sp_latest_YTD_sp_closing_high.index

# merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker #'] = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'].map(str) + ' ' + merged_portfolio_sp_latest_YTD_sp_closing_high['Counts'].map(str)

# merged_portfolio_sp_latest_YTD_sp_closing_high.head()

## YTD and Trailing Stop Charts

In [48]:
# Imports in order to be able to use Plotly offline.
!pip install plotly==4.9.0
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "colab"


print(__version__) # requires version >= 1.9.0

init_notebook_mode(connected=True)

4.9.0


In [49]:
# Ploty is an outstanding resource for interactive charts.

trace1 = go.Bar(
    x = merged_portfolio_sp_latest_YTD_sp['Ticker'][0:10],
    y = merged_portfolio_sp_latest_YTD_sp['Share YTD'][0:10],
    name = 'Ticker YTD')

trace2 = go.Scatter(
    x = merged_portfolio_sp_latest_YTD_sp['Ticker'][0:10],
    y = merged_portfolio_sp_latest_YTD_sp['SP 500 YTD'][0:10],
    name = 'SP500 YTD')
    
data = [trace1, trace2]

layout = go.Layout(title = 'YTD Return vs S&P 500 YTD'
    , barmode = 'group'
    , yaxis=dict(title='Returns', tickformat=".2%")
    , xaxis=dict(title='Ticker')
    , legend=dict(x=.8,y=1)
    )

fig = go.Figure(data=data, layout=layout)
fig.show(renderer="colab")

In [50]:
merged_portfolio_sp_latest_YTD_sp

,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close,Equiv SP Shares,SP 500 Latest Close,SP Return,Abs. Return Compare,Ticker Share Value,SP 500 Value,Abs Value Compare,Stock Gain / (Loss),SP 500 Gain / (Loss),Ticker Start Year Close,SP Start Year Close,Share YTD,SP 500 YTD,Cum Invst,Cum Ticker Returns,Cum SP Returns,Cum Ticker ROI Mult
0,AAPL,2013-02-07,125,65.40,8175.00,2019-12-31,2020-06-05,331.500000,4.068807,1509.390015,5.416095,3193.929932,1.116040,2.952767,41437.500000,17298.628544,24138.871456,33262.500000,9123.628544,292.163818,3230.780029,0.134637,-0.011406,8175.00,41437.500000,17298.628544,5.068807
6,FB,2013-12-13,150,53.32,7998.00,2019-12-31,2020-06-05,230.770004,3.328020,1775.319946,4.505103,3193.929932,0.799073,2.528947,34615.500641,14388.984727,20226.515914,26617.500641,6390.984727,205.250000,3230.780029,0.124336,-0.011406,16173.00,76053.000641,31687.613271,4.702467
1,JNJ,2014-02-27,100,81.90,8190.00,2019-12-31,2020-06-05,147.300003,0.798535,1854.290039,4.416785,3193.929932,0.722454,0.076080,14730.000305,14106.901072,623.099233,6540.000305,5916.901072,143.947876,3230.780029,0.023287,-0.011406,24363.00,90783.000946,45794.514343,3.726265
2,MCD,2013-06-04,100,84.99,8499.00,2019-12-31,2020-06-05,197.160004,1.319802,1631.380005,5.209700,3193.929932,0.957809,0.361994,19716.000366,16639.415959,3076.584407,11217.000366,8140.415959,195.080414,3230.780029,0.010660,-0.011406,32862.00,110499.001312,62433.930303,3.362516
3,MTCH,2015-12-14,600,13.63,8178.00,2019-12-31,2020-06-05,90.089996,5.609684,2021.939941,4.044631,3193.929932,0.579636,5.030048,54053.997803,12918.266486,41135.731317,45875.997803,4740.266486,82.110001,3230.780029,0.097187,-0.011406,41040.00,164552.999115,75352.196788,4.009576
4,NFLX,2016-01-14,75,108.71,8153.25,2019-12-31,2020-06-05,419.600006,2.859811,1921.839966,4.242419,3193.929932,0.661913,2.197898,31470.000458,13549.988385,17920.012073,23316.750458,5396.738385,323.570007,3230.780029,0.296783,-0.011406,49193.25,196022.999573,88902.185173,3.984754
7,TWTR,2015-01-05,225,36.38,8185.50,2019-12-31,2020-06-05,34.869999,-0.041506,2020.579956,4.051065,3193.929932,0.580700,-0.622206,7845.749760,12938.816589,-5093.066830,-339.750240,4753.316589,32.049999,3230.780029,0.087988,-0.011406,57378.75,203868.749332,101841.001763,3.553036
5,WMT,2013-08-14,125,68.30,8537.50,2019-12-31,2020-06-05,121.559998,0.779795,1685.390015,5.065593,3193.929932,0.895069,-0.115274,15194.999695,16179.149369,-984.149674,6657.499695,7641.649369,117.798302,3230.780029,0.031933,-0.011406,65916.25,219063.749027,118020.151131,3.323365


In [51]:
# Current Share Price versus Closing High Since Purchased

trace1 = go.Bar(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'][0:10],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['Pct off High'][0:10],
    name = 'Pct off High')
    
data = [trace1]

layout = go.Layout(title = 'Adj Close % off of High'
    , barmode = 'group'
    , yaxis=dict(title='% Below Adj Close High', tickformat=".2%")
    , xaxis=dict(title='Ticker')
    , legend=dict(x=.8,y=1)
    )

fig = go.Figure(data=data, layout=layout)
fig.show(renderer="colab")

## Total Return Comparison Charts

In [52]:
trace1 = go.Bar(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'][0:10],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['ticker return'][0:10],
    name = 'Ticker Total Return')

trace2 = go.Scatter(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'][0:10],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['SP Return'][0:10],
    name = 'SP500 Total Return')
    
data = [trace1, trace2]

layout = go.Layout(title = 'Total Return vs S&P 500'
    , barmode = 'group'
    , yaxis=dict(title='Returns', tickformat=".2%")
    , xaxis=dict(title='Ticker', tickformat=".2%")
    , legend=dict(x=.8,y=1)
    )

fig = go.Figure(data=data, layout=layout)
fig.show(renderer="colab")

## Cumulative Returns Over Time

In [53]:
trace1 = go.Bar(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'][0:10],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['Stock Gain / (Loss)'][0:10],
    name = 'Ticker Total Return ($)')

trace2 = go.Bar(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'][0:10],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['SP 500 Gain / (Loss)'][0:10],
    name = 'SP 500 Total Return ($)')

trace3 = go.Scatter(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'][0:10],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['ticker return'][0:10],
    name = 'Ticker Total Return %',
    yaxis='y2')

data = [trace1, trace2, trace3]

layout = go.Layout(title = 'Gain / (Loss) Total Return vs S&P 500'
    , barmode = 'group'
    , yaxis=dict(title='Gain / (Loss) ($)')
    , yaxis2=dict(title='Ticker Return', overlaying='y', side='right', tickformat=".2%")
    , xaxis=dict(title='Ticker')
    , legend=dict(x=.75,y=1)
    )

fig = go.Figure(data=data, layout=layout)
fig.show(renderer="colab")

In [54]:
trace1 = go.Bar(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['Cum Invst'],
    # mode = 'lines+markers',
    name = 'Cum Invst')

trace2 = go.Bar(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['Cum SP Returns'],
    # mode = 'lines+markers',
    name = 'Cum SP500 Returns')

trace3 = go.Bar(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['Cum Ticker Returns'],
    # mode = 'lines+markers',
    name = 'Cum Ticker Returns')

trace4 = go.Scatter(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['Cum Ticker ROI Mult'],
    # mode = 'lines+markers',
    name = 'Cum ROI Mult'
    , yaxis='y2')


data = [trace1, trace2, trace3, trace4]

layout = go.Layout(title = 'Total Cumulative Investments Over Time'
    , barmode = 'group'
    , yaxis=dict(title='Returns')
    , xaxis=dict(title='Ticker')
    , legend=dict(x=.4,y=1)
    , yaxis2=dict(title='Cum ROI Mult', overlaying='y', side='right')               
    )

fig = go.Figure(data=data, layout=layout)
fig.show(renderer="colab")